In [ ]:
# !pip install transformers -> not sure if to keep or have as prereq, for now comment out

In [1]:
from transformers import pipeline

In [2]:
summarizer = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [14]:
# sample text for development (fictional article from chatGPT)
text_file = 'sample_news_article.txt'

with open(text_file, 'r') as file:
    text_contents = file.read()

print(text_contents)

In a groundbreaking development that could reshape the future of transportation, scientists have unveiled a new generation of solar-powered cars that harness the sun's energy to drive efficiently and sustainably. The technology, showcased at the International Renewable Energy Conference held in Silicon Valley, promises to address concerns about fossil fuel dependency and reduce the carbon footprint associated with traditional vehicles.

The solar-powered cars, equipped with cutting-edge photovoltaic cells integrated into their exteriors, can convert sunlight into electricity to power the vehicle's electric motor. This innovation not only offers an eco-friendly alternative to conventional automobiles but also opens up new possibilities for cleaner and greener urban mobility.

Dr. Amanda Rodriguez, lead researcher on the project, explained, "Our goal was to create a vehicle that relies on renewable energy sources without compromising performance. By harnessing solar power, these cars can

In [20]:
# split text into sentences (split by ".", "!", and "?")
text_contents = text_contents.replace('.', ".<eos>")
text_contents = text_contents.replace('!', "!<eos>")
text_contents = text_contents.replace('?', "?<eos>")
text_contents = text_contents.replace('\n', "<eos>")

text_sentences = [sentence.strip() for sentence in text_contents.split('<eos>') if sentence.strip()]

text_sentences

["In a groundbreaking development that could reshape the future of transportation, scientists have unveiled a new generation of solar-powered cars that harness the sun's energy to drive efficiently and sustainably.",
 'The technology, showcased at the International Renewable Energy Conference held in Silicon Valley, promises to address concerns about fossil fuel dependency and reduce the carbon footprint associated with traditional vehicles.',
 "The solar-powered cars, equipped with cutting-edge photovoltaic cells integrated into their exteriors, can convert sunlight into electricity to power the vehicle's electric motor.",
 'This innovation not only offers an eco-friendly alternative to conventional automobiles but also opens up new possibilities for cleaner and greener urban mobility.',
 'Dr.',
 'Amanda Rodriguez, lead researcher on the project, explained, "Our goal was to create a vehicle that relies on renewable energy sources without compromising performance.',
 'By harnessing sol

In [29]:

# limit data sent to summarizer to 500 words 
# combine sentences into <500 word chunks without breaking any sentences up
sentence_chunks = []
max_chunk_len = 500
chunk_index = 0

for sentence in text_sentences:
    if len(sentence_chunks) == chunk_index + 1: # check if there is a current chunk
        if len(sentence_chunks[chunk_index]) + len(sentence.split(' ')) <= max_chunk_len: # if sentence can be added to current chunk do so
            sentence_chunks[chunk_index].extend(sentence.split(' ')) # add words to current chunk
        else: # create new chunk
            chunk_index += 1
            sentence_chunks.append(sentence.split(' '))
    else: # create a new chunk if there is not a current one
        sentence_chunks.append(sentence.split(' '))
        
# join indidual words back into sentences
for curr_chunk in range(len(sentence_chunks)):
    sentence_chunks[curr_chunk] = ' '.join(sentence_chunks[curr_chunk])

In [31]:
sentence_chunks[0]

'In a groundbreaking development that could reshape the future of transportation, scientists have unveiled a new generation of solar-powered cars that harness the sun\'s energy to drive efficiently and sustainably. The technology, showcased at the International Renewable Energy Conference held in Silicon Valley, promises to address concerns about fossil fuel dependency and reduce the carbon footprint associated with traditional vehicles. The solar-powered cars, equipped with cutting-edge photovoltaic cells integrated into their exteriors, can convert sunlight into electricity to power the vehicle\'s electric motor. This innovation not only offers an eco-friendly alternative to conventional automobiles but also opens up new possibilities for cleaner and greener urban mobility. Dr. Amanda Rodriguez, lead researcher on the project, explained, "Our goal was to create a vehicle that relies on renewable energy sources without compromising performance. By harnessing solar power, these cars ca

In [32]:
text_summary = summarizer(sentence_chunks, max_length = 120, min_length = 30, do_sample = False)

In [33]:
print(text_summary)

[{'summary_text': " A new generation of solar-powered cars harness the sun's energy to drive efficiently and sustainably . The technology promises to address concerns about fossil fuel dependency and reduce the carbon footprint associated with traditional vehicles . The solar cells, developed using advanced materials, are designed to be highly efficient, capturing sunlight in low-light conditions ."}]
